<a href="https://www.kaggle.com/code/yusefsavid/spaceship-titanic-1?scriptVersionId=116762996" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
from IPython.display import display_markdown
import os
display_markdown(f'''# Welcome to my {os.getenv('PYTHONPATH').split('/')[-1]} competition notebook
''', raw=True)

# Install or update any packages needed

In [ ]:
# !pip install headjackai-sdk
# !pip install --ignore-installed --pre pycaret

# Importing all that is needed

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import base64
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import gc

from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler, LabelEncoder
from sklearn.utils import shuffle
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split,HalvingGridSearchCV
from sklearn.metrics import log_loss,accuracy_score,classification_report
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.experimental import enable_halving_search_cv 
import optuna
import xgboost as xgb
from tqdm.notebook import tqdm
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, 
                              ExtraTreesClassifier, VotingClassifier)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn import datasets, linear_model
import lightgbm as lgb
from sklearn.model_selection import cross_val_score,StratifiedKFold

pd.set_option('display.max_columns', None)   

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import eli5
from eli5.sklearn import PermutationImportance
# from pycaret.classification  import *
import os

INPUT = f"/kaggle/input/{os.getenv('PYTHONPATH').split('/')[-1]}/"
OUTPUT = "/kaggle/working/"


# Reading all datasets and some light analysis

In [ ]:
train =  pd.read_csv(f"{INPUT}/train.csv")#.sample(frac=1).reset_index(drop=True)
test = pd.read_csv(f"{INPUT}/test.csv")#.sample(frac=1).reset_index(drop=True)
sample = pd.read_csv(f"{INPUT}/sample_submission.csv")
traintest = pd.concat([train,test], axis = 0).reset_index(drop=True)

In [ ]:
traintest

In [ ]:
traintest.describe()

In [ ]:
traintest.info()

In [ ]:
sample 

## split columns that could be useful split

In [ ]:
splitable = ["PassengerId" ,"Cabin", "Name"]
delimeters = ["_" , "/" , "-"," "]
# traintest = pd.concat([train,test], axis = 0).sample(frac=1).reset_index(drop=True)
for j,i in enumerate(splitable):
    traintest = pd.concat([traintest,traintest[i].str.split("|".join(delimeters),expand=True).add_prefix(f'{i}_')], axis = 1)
#     test = pd.concat([test,test[i].str.split(delimeters[j],expand=True).add_prefix(f'{i}_')], axis = 1) 
#     traintest.drop(i, axis= 1, inplace = True)
#     test.drop(i, axis= 1, inplace = True)
# traintest[list(set(traintest.columns) - set(["Transported"])) ] = traintest[list(set(traintest.columns) - set(["Transported"])) ].fillna(method = "bfill")

## Create new columns 

In [ ]:
traintest["Expenses"] = traintest[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck' ]].sum(axis = 1)

## CryoSleep

Check if there are rows where expenses is more than 0.0 while CryoSleep being True(should not happen)

In [ ]:
traintest.query("CryoSleep == True and Expenses > 0.0")

In [ ]:
traintest.query("Age <= 13.0").CryoSleep.value_counts().plot.pie()

In [ ]:
traintest.query("Age > 13.0").CryoSleep.value_counts().plot.pie()

In [ ]:
traintest.query("Age == 0.0").Expenses.hist()
print(f"The maximum expense for those with age 0 is :{traintest.query('Age == 0.0').Expenses.max()}")

In [ ]:
sns.countplot(traintest.query("Age == 0.0").CryoSleep)

In [ ]:
traintest.Age.hist()
print(f"The median is {traintest.Age.median()}")

In [ ]:
traintest.query("Expenses > 0.0").Age.hist()
plt.show()
print(f"The median is {traintest.query('Expenses > 0.0').Age.median()}")
print(f"The minimum is {traintest.query('Expenses > 0.0').Age.min()}")

In [ ]:
traintest.query("CryoSleep == False and Expenses == 0.0").Age.hist(bins = 80)
plt.axvline(13 ,c = "r")
plt.show()
print(f"The median is {traintest.query('CryoSleep == False and Expenses == 0.0').Age.median()}")

Seems like those under 13 are not allowed to spend, or somehow pay with their parents/guardians.
We can fill all those with zero expenses with nan Age to the median of 7.0. For those with nan as age but with expenses > 0, we can fill with median of 28.

In [ ]:
traintest.loc[traintest["Age"].isnull() == True]

In [ ]:
plt.scatter(traintest.query('Expenses > 0.0').Expenses.values , traintest.query('Expenses > 0.0').Age.values)

## VIP

In [ ]:
sns.countplot(traintest.VIP)


In [ ]:
traintest.query("VIP == True").describe()

In [ ]:
traintest.query("VIP == False").describe()

Clearly those with VIP passes are people who spend more. We can use the to fill the VIP column. VIPs also seem to be generally older individuals Expenses <1400 and Age <37 not VIP. We could even fill all with False for VIP as it is heavily skewed towards being False. 

### Cabins

In [ ]:
traintest[traintest["Cabin"].isnull() == True].Destination.value_counts()

In [ ]:
pd.pivot_table(traintest, values='Age', index=['Destination'],
                    columns=['Cabin_0'], aggfunc="count")

In [ ]:
pd.pivot_table(traintest, values='PassengerId', index=['Destination'],
                    columns=['Cabin_2'], aggfunc="count")

In [ ]:
pd.set_option("display.max_rows", 50)
traintest["PassengerId0_repeated"] = ""
traintest.PassengerId0_repeated = traintest.PassengerId_0.map(traintest.groupby('PassengerId_0')["PassengerId_0"].count() )
# traintest.query("PassengerId0_repeated > 1 ").groupby('PassengerId_0').agg(','.join)
traintest[(traintest["Cabin"].isnull() == True) & traintest["PassengerId0_repeated"] > 1]

# pd.set_option("display.max_rows", 20)

Seems like PassengerId wouldnt help impute Cabin nans. Cabin_0 filled with F if TRAPPIST-1e , G if PSO J318.5-22 , F if 55 Cancri e , else F could be used


## Destination

In [ ]:
traintest[(traintest["Destination"].isnull() == True) & traintest["PassengerId0_repeated"].isnull() > 1 ]

In [ ]:
traintest[(traintest["Destination"].isnull() == True)]

In [ ]:
pd.pivot_table(traintest, values='PassengerId', index=['Cabin_0'],
                    columns=['Destination'], aggfunc="count")

In [ ]:
pd.pivot_table(traintest, values='PassengerId', index=['Cabin_2'],
                    columns=['Destination'], aggfunc="count")

In [ ]:
pd.pivot_table(traintest, values='PassengerId', index=['HomePlanet'],
                    columns=['Destination'], aggfunc="count")

Seems like no quick rules to create to impute **Destination**

### HomePlanet

In [ ]:
traintest[(traintest["HomePlanet"].isnull() == True)]

In [ ]:
pd.pivot_table(traintest, values='PassengerId', index=['Cabin_2'],
                    columns=['HomePlanet'], aggfunc="count")

In [ ]:
pd.pivot_table(traintest, values='PassengerId', index=['Cabin_0'],
                    columns=['HomePlanet'], aggfunc="count")

In [ ]:
pd.pivot_table(traintest, values='PassengerId', index=['Destination'],
                    columns=['HomePlanet'], aggfunc="count")

In [ ]:
pd.pivot_table(traintest, values='PassengerId', index=['VIP'],
                    columns=['HomePlanet'], aggfunc="count")

In [ ]:
mapper_homeplant = {"A":"Europa" , "B":"Europa","C":"Europa" ,"G":"Earth" , "T":"Europa"}
# mapper_homeplant_1 = {False: "Earth" , True:"Europa"}

Seems like we can make some kind of dict to map by using Cabin_0 to impute HomePlanet. We can also see that Earthlings do not get a VIP pass often, however am not gonna create any rules based on this fact and ill let the imputer later impute missing values here.

Check if persons from other plants are on average older or if they spend more

In [ ]:
traintest[(traintest["HomePlanet"].isnull() == True)].Age.hist()
plt.show()
print(f"Median age of nans:{traintest[(traintest['HomePlanet'].isnull() == True)].Age.median()}")
print(f"Median age of non nans:{traintest.Age.median()}")

In [ ]:
traintest[(traintest["HomePlanet"].isnull() == True)].Expenses.hist()
print(f"Median expenses of nans:{traintest[(traintest['HomePlanet'].isnull() == True)].Expenses.median()}")
print(f"Median expenses of non nans:{traintest.Expenses.median()}")

In [ ]:
traintest[(traintest["HomePlanet"].isnull() == True) & traintest["PassengerId0_repeated"].isnull() > 1 ]

So lets summerize the missing value imputation rules we created so far:
- If Expenses = 0 then CryoSleep = True else False
- If Expenses = 0 then Age = 7. If expenses > 0 then Age = 27 
- If Expenses < 1400 & Age < 37 then not VIP Else VIP
- Use mapper_homeplanet to Impute HomePlanet using Cabin_0.

In [ ]:
traintest

In [ ]:
def cryosleep_nan(row):
    if row['Expenses'] == 0.0:
        return True
    else: 
        return False

traintest.loc[traintest["CryoSleep"].isnull() == True , "CryoSleep"] = traintest.apply(lambda row:cryosleep_nan(row), axis=1 )
traintest.loc[(traintest["Expenses"]== 0.0)&(traintest["Age"].isnull()) , "Age"] = traintest.query('CryoSleep == False and Expenses == 0.0').Age.median()
traintest.loc[(traintest["Expenses"] > 0.0) &(traintest["Age"].isnull())  ,"Age"] = traintest.query('Expenses > 0.0').Age.median()
traintest.loc[(traintest["Expenses"]<1400)&(traintest["Age"]<37)&(traintest["VIP"].isnull())  , "VIP"] = False
traintest.loc[traintest["VIP"].isnull() , "VIP"] = True
traintest.loc[traintest["HomePlanet"].isnull(), "HomePlanet"] = traintest.Cabin_0.map(mapper_homeplant)
traintest["cabin_count"] = traintest.Cabin_1.map(traintest.groupby("Cabin_1")["Cabin_1"].count())
traintest["passengerId_count"] = traintest.PassengerId_0.map(traintest.groupby("PassengerId_0")["PassengerId_0"].count() ) 
traintest.loc[traintest.Expenses == 0.0, "did_spend"]  = False
traintest.loc[traintest.Expenses > 0.0 , "did_spend"] = True 
traintest.loc[traintest.Age == 0.0 , "age_zero"] = True
traintest.loc[traintest.Age > 0.0 , "age_zero"] = False
traintest.loc[traintest.Age <13.0 , "is_minor"] = True
traintest.loc[traintest.Age >= 13.0, "is_minor"] = False 
traintest["age_bin"] = pd.cut(traintest["Age"], bins=10, labels=False)
# traintest.loc[(traintest["Age"] == 0.0) & (traintest["Expenses"] == 0.0) ,"Age"] = traintest.query('CryoSleep == False and Expenses == 0.0').Age.median()


In [ ]:
traintest.info()

In [ ]:
traintest

In [ ]:
# features = test.columns ### For now assuming all column names other than the target are features
target = "Transported" ### Change to the column name of the target variable
numeric_features = ["Age", 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck' , "PassengerId_0", "Expenses"  ,"cabin_count" ,"passengerId_count"  ] ### Add all numeric features
categorical_features = ["HomePlanet" , "CryoSleep" ,"Destination" ,"VIP","PassengerId_1" ,"Cabin_0" , "Cabin_2" ,"did_spend" ,"age_zero" , "is_minor" , "age_bin"]
traintest[numeric_features] = traintest[numeric_features].astype("float")

## EDA

In [ ]:
plt.figure(figsize = (15,15))
sns.heatmap(traintest[numeric_features].corr(),annot = True) ### check for any high values, will be addressed later

In [ ]:
for i in numeric_features: ### Check for skwenness 
    plt.title(f"{i}")
    plt.hist(traintest[f"{i}"])
    plt.show()
remove_skewness =[ 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck' , "Expenses" ]

In [ ]:
for i in list(categorical_features):
    plt.title(i)
    sns.countplot(x = traintest[f"{i}"].values)
    plt.show()

# Time to work on the data!!

### Fix skeweness of numeric data

In [ ]:
for i in remove_skewness:
    traintest[i] = np.sqrt(traintest[i])

In [ ]:
for i in numeric_features: ### Check for skwenness 
    plt.title(f"{i}")
    plt.hist(traintest[f"{i}"])
    plt.show()
remove_skewness =[ 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck' , "Expenses" ]

In [ ]:
## check if target is skewed

In [ ]:
# imputer = SimpleImputer(strategy = "mean")
# traintest.loc[:,numeric_features] = imputer.fit_transform(traintest.loc[:,numeric_features].values)
# # test.loc[:,numeric_features] = imputer.transform(test.loc[:,numeric_features].values)
# imputer = SimpleImputer(strategy = "most_frequent")
# traintest.loc[:,categorical_features] = imputer.fit_transform(traintest.loc[:,categorical_features].values)
# # test.loc[:,categorical_features]= imputer.fit_transform(test.loc[:,categorical_features].values)

### Check colinearity 

In [ ]:
# #check colinearity
# #Compute VIF data for each independent variable
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# vif = pd.DataFrame()
# vif["features"] = traintest[numeric_features].columns
# vif["vif_Factor"] = [variance_inflation_factor(traintest[numeric_features].values, i) for i in range(traintest[numeric_features].shape[1])]
# vif
# #>10 not so good

In [ ]:
# #feature engineer
# from headjackai.headjackai_hub import headjackai_hub

# #host setting
# hj_hub = headjackai_hub('http://www.headjackai.com:9000')
# hj_hub.login(username='jimliu_kaggle', pwd='12345678')
# hj_hub.knowledgepool_check(public_pool=True)
# source = 'drug_type'


# best_features = [
#  'Spaceship-Titanic-CryoSleep',
#  'Spaceship-Titanic-RoomService',
#  'Spaceship-Titanic-Spa',
#  'Spaceship-Titanic-VRDeck',
#  'Spaceship-Titanic-deck',
#  'Spaceship-Titanic-side',
#  'Spaceship-Titanic-SumSpends',
#  'drug_type-Sex']


# # run features engineering on hj 
# hj_X = hj_hub.knowledge_transform(data=train, target_domain='Spaceship-Titanic', 
#                                   source_domain=source,
#                                   label='')

# hj_ts_X = hj_hub.knowledge_transform(data=test, target_domain='Spaceship-Titanic', 
#                                   source_domain=source,
#                                   label='') 

# Encode, impute missing values and scale

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
labenc = LabelEncoder()
labenc.fit(traintest.loc[traintest[target].isnull() == False ,target].unique())

traintest.loc[traintest[target].isnull() == False ,target] = \
labenc.transform(traintest.loc[traintest[target].isnull() == False ,target])
for i in categorical_features:
    enc = LabelEncoder()
    traintest.loc[traintest[i].isnull() == False, i] = enc.fit_transform(traintest.loc[traintest[i].isnull() == False, i].values)
imputer = KNNImputer()
traintest[numeric_features+categorical_features] = imputer.fit_transform(traintest[numeric_features+categorical_features].values)
scaler = StandardScaler()
traintest[numeric_features+categorical_features] = scaler.fit_transform(traintest[numeric_features+categorical_features].values)

### Feature engineering time 

# Time for some ML

In [ ]:
# features_isolation =numeric_features+categorical_features
# from sklearn.ensemble import IsolationForest

# isf = IsolationForest(n_jobs=-1, random_state=1,n_estimators=100,contamination=0.002)
# isf.fit(traintest.loc[features_isolation], traintest[target])

# rows = pd.DataFrame(isf.predict(traintest[features_isolation]),columns=['feature'])
# rows_ind = rows[rows.feature == 1]
# results = pd.DataFrame()
# results['results'] = - isf.score_samples(traintest[features_isolation])
# results['chance'] = isf.decision_function(traintest[features_isolation])
# results['yes/no'] = isf.predict(traintest[features_isolation])

# new = results[results['yes/no']==1]
# old= results[results['yes/no']==-1]
# fig,ax = plt.subplots(figsize=(20,10))
# ax.set_facecolor('black')
# ax = plt.scatter(new.index,new.results,marker=".")
# ax = plt.scatter(old.index,old.results,color='white',marker="*")

## split into features and targets for the models

In [ ]:
x = traintest.loc[traintest["Transported"].isnull() == False  , numeric_features +categorical_features].values
y = traintest.loc[traintest["Transported"].isnull() == False  , target].values
x_test = traintest.loc[traintest["Transported"].isnull() == True  , numeric_features +categorical_features].values
y=y.astype('int')
x = x.astype("float64")
x_test = x_test.astype("float64")
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

## Select all the models needed

In [ ]:
import xgboost as xgb
params_XGB_best= {'lambda': 3.0610042624477543, 
             'alpha': 4.581902571574289, 
             'colsample_bytree': 0.9241969052729379, 
             'subsample': 0.9527591724824661, 
             'learning_rate': 0.06672065863100594, 
             'n_estimators': 725, #initial value is 651
             'max_depth': 5, 
             'min_child_weight': 1, 
             'num_parallel_tree': 1}
classifiers = [

RandomForestClassifier(),
AdaBoostClassifier(),
GradientBoostingClassifier(),
ExtraTreesClassifier(),
LinearDiscriminantAnalysis(),
LogisticRegression(),
KNeighborsClassifier(),
DecisionTreeClassifier(),
MLPClassifier(),
CatBoostClassifier(verbose = False),
SVC()
]

## Running through all and cross validating  

In [ ]:
model_results_columns = ['fit_time', 'score_time', 'test_accuracy', 'train_accuracy',
       'test_precision', 'train_precision', 'test_recall', 'train_recall',
       'test_r2', 'train_r2', 'test_neg_mean_squared_error',
       'train_neg_mean_squared_error']
model_results_df = pd.DataFrame([], columns = model_results_columns)
for clf in tqdm(classifiers):
    model = cross_validate(clf , x, y , cv= 10,\
                           scoring=('accuracy' , 'precision', 'recall' , 'r2', 'neg_mean_squared_error'),return_train_score=True)
    model_results_df = pd.concat([model_results_df,pd.DataFrame(pd.DataFrame(model).describe().loc["mean" ,:]).rename(columns = {"mean":str(clf).split("(")[0]}).transpose()], axis = 0)

## Analyse results
For the **overfitting** column, the closer to zero the better

In [ ]:
model_results_df["overfitting"] =(1-(model_results_df["test_accuracy"]/ model_results_df["train_accuracy"]))*100
model_results_df.sort_values("test_accuracy", ascending = False)

## Combine models

In [ ]:
# model = VotingClassifier([('cbc',CatBoostClassifier(verbose = False)) , ('svc',SVC()),('gbc',GradientBoostingClassifier())] , voting = "hard")

In [ ]:

# re = cross_validate(model , x, y , cv= 10,\
#                            scoring=('accuracy' , 'precision', 'recall' , 'r2', 'neg_mean_squared_error'),return_train_score=True)

## Hyperparameter tuning

In [ ]:
hp = {
    "criterion":['gini', 'entropy'],
    "splitter":["best" ,"random"],
    "min_samples_split":[1.0,2,3,4]
}
# hp = {
#     "hidden_layer_sizes":[50,100,200,300,1000],
#     "activation":['identity', 'logistic', 'tanh', 'relu'],
#     "solver":["lbfgs" , 'sgd','adam'],
#     "alpha":[0.0001,0.001,0.01,0.00001],
#     "learning_rate":["constant" , "invscaling" ,"adapting"],
#     "learning_rate_init":[0.1,0.01,0.001,0.0001],
#     "max_iter":[20000]
    
# }
params_RandomForestClassifier = {
    "n_estimators":[10,100,200,400],
#     "criterion":['gini', 'entropy'],
#     "max_depth":[100,10,None],
#     "max_features":["sqrt" , "log2" , None],
}
params_catboost = {'depth'         : [4,5,6,7,8,9, 10],
                 'learning_rate' : [0.01,0.02,0.03,0.04],
                  'iterations'    : [10, 20,30,40,50,60,70,80,90, 100]
                 }    




In [ ]:
# from sklearn.experimental import enable_halving_search_cv 
# from sklearn.model_selection import HalvingGridSearchCV
# sh = HalvingGridSearchCV(model, params_catboost, cv=10,verbose = 2, scoring = "accuracy").fit(x, y)

# Choose the best model to fit and then predict on test

In [ ]:
model = SVC()

In [ ]:
re = pd.DataFrame(cross_validate(model , x, y , cv= 10,\
                           scoring=('accuracy' , 'precision', 'recall' , 'r2', 'neg_mean_squared_error'),return_train_score=True))

In [ ]:
re.test_accuracy.mean()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
preds = model.predict(x_test) 

In [ ]:
pd.concat([test.reset_index() , pd.DataFrame(labenc.inverse_transform(preds), columns = ["Transported"])], axis = 1)[["PassengerId", "Transported"]]

# Download submission

In [ ]:
pd.concat([test.reset_index() , pd.DataFrame(labenc.inverse_transform(preds), columns = ["Transported"])], axis = 1)[["PassengerId", "Transported"]].to_csv("submission.csv", index = False)